# Импорт библиотек

In [796]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')




## Начало анализа

В этом датасете содержится комплексное исследование по поведению во время ежедневных поездок. Целью этого проекта является прогнозирование предпочтительного способа передвижения (например, автомобиль, велосипед, общественный транспорт) на основе личных и бытовых характеристик, включая местоположение рабочего места и дома

У нас таргет -- KHVM, используемое средство передвижения, задача классфикации, всего кклассов 7

In [797]:
df = pd.read_csv("Dataset_2_DATA.csv")
display(df)
print(df.iloc[0, :])




,HHID,PERSID,VPLID,RitID,HH_VALID,P_VALID,KHVM,WEGGEWEEST,VERTREKP,AANTVPL,...,HHAUTO_N,HHAUTO,HHBESTEL,HHHYBRID,HHMOTOR,HHBROM,HHSNOR,HHFIETS,HHVOUWFIETS,HHEBIKE
0,30055622,3005562201,13957101,1395710101,2,3,2,1,2,2,...,3,1,0,0,1,0,0,1,0,1
1,30055622,3005562201,13957102,1395710201,2,3,2,1,2,2,...,3,1,0,0,1,0,0,1,0,1
2,30055622,3005562201,13957201,1395720101,2,3,1,1,2,4,...,3,1,0,0,1,0,0,1,0,1
3,30055622,3005562201,13957202,1395720201,2,3,1,1,2,4,...,3,1,0,0,1,0,0,1,0,1
4,30204076,3020407601,14445303,1444530301,2,3,8,1,1,3,...,2,1,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,30862294,3086229401,19042304,1904230401,2,3,2,1,2,12,...,1,1,0,0,0,0,0,1,0,0
7306,30862294,3086229401,19042103,1904210301,2,3,7,1,2,5,...,1,1,0,0,0,0,0,1,0,0
7307,30862294,3086229401,19042104,1904210401,2,3,7,1,2,5,...,1,1,0,0,0,0,0,1,0,0
7308,30862294,3086229401,19042201,1904220101,2,3,1,1,2,3,...,1,1,0,0,0,0,0,1,0,0


HHID                         30055622
PERSID                     3005562201
VPLID                        13957101
RitID                      1395710101
HH_VALID                            2
P_VALID                             3
KHVM                                2
WEGGEWEEST                          1
VERTREKP                            2
AANTVPL                             2
VPLDAGNR                            1
VERPL                               3
VERPLNR                             1
TOER                                0
TOER_TYPE                           9
AANTRIT                             1
KMOTIEF                             2
VERTPROV                            0
AANKPROV                            0
KAFSTV                             13
KVERTTIJD                           8
KREISDUUR                          11
ROLAUTO                             2
PARKEERKOSTEN                       0
VERTRAGING                          0
DAGSOORT                            1
OORDEEL_AUTO

# Подготовка данных

У нас есть пустые значения, попробуем понять, что это за значения и как их можно дополнить или удалить вовсе

## Обработка пропусков и дубликатов

In [798]:
df = df.replace({' ': np.nan})

print("PERSID = 3077839202")
display(df[df['PERSID'] == 3077839202][['PERSID', 'KREISDUUR', 'PARKEERKOSTEN']])


nan_rows = df[df.isna().any(axis=1)]
display(nan_rows[['PERSID', 'KREISDUUR', 'PARKEERKOSTEN']])

# ---- Краткая справка ----
# 'KREISDUUR': 'класс_длительности_поездки'
# 'PARKEERKOSTEN': 'класс_стоимости_парковки'

# Функция для заполнения пропусков наиболее частым значением
def fill_with_mode(group):
    return group.fillna(group.mode().iloc[0] if not group.mode().empty else group.mean())

cols_to_fill = ['KREISDUUR', 'PARKEERKOSTEN']

for col in cols_to_fill:
    df[col] = df.groupby('PERSID')[col].transform(fill_with_mode)

print("Проверка заполненных значений для PERSID = 3077839202")
display(df[df['PERSID'] == 3077839202][cols_to_fill])

# Обработка дубликатов

df = df.drop_duplicates()


PERSID = 3077839202


,PERSID,KREISDUUR,PARKEERKOSTEN
169,3077839202,NaN,0
207,3077839202,3,0
352,3077839202,4,0
1548,3077839202,3,NaN
1867,3077839202,3,0
1868,3077839202,3,0
1869,3077839202,5,0
1871,3077839202,7,0
7062,3077839202,8,0


,PERSID,KREISDUUR,PARKEERKOSTEN
169,3077839202,NaN,0
256,3030794001,NaN,0
257,3030794001,NaN,0
258,3030794001,NaN,0
259,3030794001,NaN,0
...,...,...,...
6800,3005421901,NaN,0
6813,3058987802,NaN,0
6815,3005421901,NaN,0
6854,3085101701,NaN,0


Проверка заполненных значений для PERSID = 3077839202


,KREISDUUR,PARKEERKOSTEN
169,3,0
207,3,0
352,4,0
1548,3,0
1867,3,0
1868,3,0
1869,5,0
1871,7,0
7062,8,0


## Приводим колонки к типу int

In [799]:
print(df['KREISDUUR'].value_counts().reset_index())
print(df['PARKEERKOSTEN'].value_counts().reset_index())


print(df.info())

df['KREISDUUR'] = df['KREISDUUR'].astype('int64')
df['PARKEERKOSTEN'] = df['PARKEERKOSTEN'].astype('int64')

print(df.info())

   KREISDUUR  count
0          2   1301
1          3   1206
2          7   1029
3          4    879
4          9    602
5          8    498
6          5    469
7          1    412
8          6    344
9         11    289
10        10    281
   PARKEERKOSTEN  count
0              0   7207
1              5     16
2              9     12
3             16     10
4             13      9
5              2      9
6             10      7
7              6      7
8              7      7
9              4      5
10             3      5
11            11      5
12            12      4
13            14      4
14             1      2
15             8      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7310 entries, 0 to 7309
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   HHID                     7310 non-null   int64 
 1   PERSID                   7310 non-null   int64 
 2   VPLID                    731

## Применяем дополнительное условие к 7 варианту

In [800]:
df = df[df['KAFSTV'] >= 4] # по условию 7 варианта "Дополнительные условия:KAFSTV(Distance class trip) больше 3.5 км"
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4546 entries, 0 to 7309
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   HHID                     4546 non-null   int64
 1   PERSID                   4546 non-null   int64
 2   VPLID                    4546 non-null   int64
 3   RitID                    4546 non-null   int64
 4   HH_VALID                 4546 non-null   int64
 5   P_VALID                  4546 non-null   int64
 6   KHVM                     4546 non-null   int64
 7   WEGGEWEEST               4546 non-null   int64
 8   VERTREKP                 4546 non-null   int64
 9   AANTVPL                  4546 non-null   int64
 10  VPLDAGNR                 4546 non-null   int64
 11  VERPL                    4546 non-null   int64
 12  VERPLNR                  4546 non-null   int64
 13  TOER                     4546 non-null   int64
 14  TOER_TYPE                4546 non-null   int64
 15  AANTRIT  

None

## Проверка выбросов и стандартизация числовых переменных

In [801]:
box_fig = px.box(df[['HHPERS', 'N_KIND', 'AANTVPL', 'AANTRIT', 'HHAUTO_N']])
box_fig.update_layout(title = {'text': 'Boxplot-ы для числовых переменных', 'x': 0.5})



Такие выбросы мы оставим, так как тут все outliers можно как-то объяснить и вывести на практическую плоскость

В конечном итоге мы получим готовую витрину `df`, которая будет исспользоваться в построении моделей машинного общения 

In [802]:
display(df)

,HHID,PERSID,VPLID,RitID,HH_VALID,P_VALID,KHVM,WEGGEWEEST,VERTREKP,AANTVPL,...,HHAUTO_N,HHAUTO,HHBESTEL,HHHYBRID,HHMOTOR,HHBROM,HHSNOR,HHFIETS,HHVOUWFIETS,HHEBIKE
0,30055622,3005562201,13957101,1395710101,2,3,2,1,2,2,...,3,1,0,0,1,0,0,1,0,1
1,30055622,3005562201,13957102,1395710201,2,3,2,1,2,2,...,3,1,0,0,1,0,0,1,0,1
2,30055622,3005562201,13957201,1395720101,2,3,1,1,2,4,...,3,1,0,0,1,0,0,1,0,1
3,30055622,3005562201,13957202,1395720201,2,3,1,1,2,4,...,3,1,0,0,1,0,0,1,0,1
4,30204076,3020407601,14445303,1444530301,2,3,8,1,1,3,...,2,1,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7303,30862294,3086229401,19042203,1904220301,2,3,6,1,2,3,...,1,1,0,0,0,0,0,1,0,0
7304,30862294,3086229401,19042102,1904210201,2,3,1,1,2,5,...,1,1,0,0,0,0,0,1,0,0
7305,30862294,3086229401,19042304,1904230401,2,3,2,1,2,12,...,1,1,0,0,0,0,0,1,0,0
7308,30862294,3086229401,19042201,1904220101,2,3,1,1,2,3,...,1,1,0,0,0,0,0,1,0,0


Приведем описание готовой витрины

## Интепретирование данных


**Базовые идентификаторы**
* 'HHID': 'id_домохозяйства',
* 'PERSID': 'id_человека',
* 'VPLID': 'id_поездки',
* 'RitID': 'id_участка_поездки',

**Валидация данных**
* 'HH_VALID': 'статус_валидации_домохозяйства',
* 'P_VALID': 'статус_валидации_человека',

**Характеристики поездок (основной фокус исследования)**

* 'KHVM': 'основной_транспорт',
* 'WEGGEWEEST': 'факт_поездки_в_день',
* 'VERTREKP': 'точка_отправления_первой_поездки',
* 'AANTVPL': 'количество_поездок',
* 'VPLDAGNR': 'номер_дня_поездки',
* 'VERPL': 'тип_поездки',
* 'VERPLNR': 'порядковый_номер_поездки',
* 'TOER': 'флаг_круговой_поездки',
* 'TOER_TYPE': 'тип_круговой_поездки',
* 'AANTRIT': 'количество_сегментов_поездки',
  
**Цели поездок**
* 'KMOTIEF': 'цель_поездки',
 
**География**
* 'VERTPROV': 'провинция_отправления',
* 'AANKPROV': 'провинция_прибытия',
  
**Характеристики маршрута (важно для условия >3.5 км)**

* 'KAFSTV': 'класс_расстояния_поездки',
* 'KVERTTIJD': 'класс_времени_отправления',
* 'KREISDUUR': 'класс_длительности_поездки',
  
**Использование автомобиля**

* 'ROLAUTO': 'роль_в_автомобиле',
* 'PARKEERKOSTEN': 'класс_стоимости_парковки',
* 'VERTRAGING': 'класс_задержки_поездки',
  
**Временные характеристики**

* 'DAGSOORT': 'тип_дня',
  
**Оценки видов транспорта (ключевой контекст)**

* 'OORDEEL_AUTO': 'оценка_автомобиля',
* 'OORDEEL_TREIN': 'оценка_поезда',
* 'OORDEEL_BTM': 'оценка_общественного_транспорта',
* 'OORDEEL_FIETS': 'оценка_велосипеда',
* 'OORDEEL_BROMMER': 'оценка_мопеда',
* 'OORDEEL_LOPEN': 'оценка_передвижения_пешком',
  
**Восприятие транспорта (важно для моделирования предпочтений)**
  
* 'GEBRUIK_AUTO_STELLING3': 'восприятие_экономии_времени_авто',
* 'GEBRUIK_AUTO_STELLING4': 'восприятие_безопасности_авто',
* 'GEBRUIK_TREIN_STELLING3': 'восприятие_экономии_времени_поезда',
* 'GEBRUIK_TREIN_STELLING4': 'восприятие_безопасности_поезда',
* 'GEBRUIK_BTM_STELLING3': 'восприятие_экономии_времени_общественного_транспорта',
* 'GEBRUIK_BTM_STELLING4': 'восприятие_безопасности_общественного_транспорта',
* 'GEBRUIK_FIETS_STELLING2': 'восприятие_релаксации_на_велосипеде',
* 'GEBRUIK_FIETS_STELLING3': 'восприятие_экономии_времени_велосипеда',
* 'GEBRUIK_LOPEN_STELLING3': 'восприятие_экономии_времени_ходьбы',
* 'GEBRUIK_LOPEN_STELLING4': 'восприятие_безопасности_ходьбы',
  
**Характеристики домохозяйства (ключевые предикторы)**

* 'HHPERS': 'количество_в_домохозяйстве',
* 'HHSAM': 'состав_домохозяйства',
* 'N_KIND': 'дети_до12_в_домохозяйстве',
* 'HHBRUTOINK2_w5': 'класс_дохода_домохозяйства',
* 'HHAUTO_N': 'количество_автомобилей',
* 'HHAUTO': 'наличие_автомобиля',
* 'HHBESTEL': 'наличие_фургона',
* 'HHHYBRID': 'наличие_гибридного_авто',
* 'HHMOTOR': 'наличие_мотоцикла',
* 'HHBROM': 'наличие_мопеда',
* 'HHSNOR': 'наличие_скутера',
* 'HHFIETS': 'наличие_велосипеда',
* 'HHVOUWFIETS': 'наличие_складного_велосипеда',
* 'HHEBIKE': 'наличие_электровелосипеда'
  
  

# Анализ (EDA)

Теперь расшифруем значения, которые есть у нас в датасете, чтобы мы смогли построить несколько наглядных графиков, не заглядывая в описание датасета и смотреть на числа и мысленно подставлять категории

In [803]:
desc = pd.read_csv('Dataset_2_DESCRIPTION.csv', encoding='latin1')
display(desc) 

,Variable,Var_description,Value,Label,Measurement_level,BaseData_Type,Name_BaseData,Filter_Variables
0,PERSOON,new person,0,no new person,Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1
1,PERSOON,new person,1,new person,Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1
2,HHID,Unique Household ID,999999999,Input,Scale,Mobility,MPNWAVE6_DAGBOEKdata,1
3,PERSID,Unique ID of Person,999999999,Unique ID of Person,Scale,Mobility,MPNWAVE6_DAGBOEKdata,1
4,HH_VALID,Complete household yes or no,0,"Not a complete household, not all persons comp...",Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1
...,...,...,...,...,...,...,...,...
1054,tramhalte,Straight line distance between nearest tram st...,999999999,Straight line distance between nearest tram st...,Scale,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,0
1055,bushalte4xpu,Straight line distance between nearest bus sto...,999999999,Straight line distance between nearest bus sto...,Scale,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,0
1056,bushalte2xpu,Straight line distance between nearest bus sto...,999999999,Straight line distance between nearest bus sto...,Scale,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,0
1057,bushalte1xpu,Straight line distance between nearest bus sto...,999999999,Straight line distance between nearest bus sto...,Scale,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,0


Расшифровка и интепретация написанного

In [804]:
needed_description = pd.merge(desc, pd.DataFrame(df.columns), left_on='Variable', right_on=0)
display(needed_description)

,Variable,Var_description,Value,Label,Measurement_level,BaseData_Type,Name_BaseData,Filter_Variables,0
0,HHID,Unique Household ID,999999999,Input,Scale,Mobility,MPNWAVE6_DAGBOEKdata,1,HHID
1,PERSID,Unique ID of Person,999999999,Unique ID of Person,Scale,Mobility,MPNWAVE6_DAGBOEKdata,1,PERSID
2,HH_VALID,Complete household yes or no,0,"Not a complete household, not all persons comp...",Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1,HH_VALID
3,HH_VALID,Complete household yes or no,1,"Not a complete household, but all persons comp...",Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1,HH_VALID
4,HH_VALID,Complete household yes or no,2,Yes a complete household,Nominal,Mobility,MPNWAVE6_DAGBOEKdata,1,HH_VALID
...,...,...,...,...,...,...,...,...,...
303,HHEBIKE,"Yes or no electric bicycle, e-bike in the hous...",0,No,Nominal,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,1,HHEBIKE
304,HHEBIKE,"Yes or no electric bicycle, e-bike in the hous...",1,Yes,Nominal,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,1,HHEBIKE
305,HHEBIKE,"Yes or no electric bicycle, e-bike in the hous...",99,"no household questionnaire, no imputation from...",Nominal,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,1,HHEBIKE
306,HHPERS,Number of people in the household,999999999,Number of people in the household,Nominal,Data_HouseHold,MPNWAVE6_HHdata_North-Holland,1,HHPERS


Создание понятных значений и ренейм столбцов для понимания

In [805]:
df_map = df.copy()

# Создаем словарь вида {колонка: {значение: метка}}
mapping = {}
for var in needed_description['Variable'].unique():
    subset = needed_description[needed_description['Variable'] == var]
    mapping[var] = dict(zip(subset['Value'], subset['Label']))

for column in df.columns:
    if column in mapping:
        # Конвертируем ключи маппинга в строки для совместимости с разными типами данных
        str_map = {str(k): v for k, v in mapping[column].items()}
        df_map[column] = df_map[column].astype(str).replace(str_map)

id_columns = ['HHID', 'PERSID', 'VPLID', 'RitID']
for col in id_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df_map[col], errors='coerce')

In [806]:
display(df_map.iloc[0, :])
print('-' * 100)
display(df.iloc[0, :])

HHID                                                                30055622
PERSID                                                            3005562201
VPLID                                                               13957101
RitID                                                             1395710101
HH_VALID                                            Yes a complete household
P_VALID                    person completed the questionnaire and complet...
KHVM                                                        Car as passenger
WEGGEWEEST                           On this day the person did go somewhere
VERTREKP                                                     Another address
AANTVPL                                                                    2
VPLDAGNR                                         first day of recorded trips
VERPL                                                            trip abroad
VERPLNR                                                                    1

----------------------------------------------------------------------------------------------------


HHID                         30055622
PERSID                     3005562201
VPLID                        13957101
RitID                      1395710101
HH_VALID                            2
P_VALID                             3
KHVM                                2
WEGGEWEEST                          1
VERTREKP                            2
AANTVPL                             2
VPLDAGNR                            1
VERPL                               3
VERPLNR                             1
TOER                                0
TOER_TYPE                           9
AANTRIT                             1
KMOTIEF                             2
VERTPROV                            0
AANKPROV                            0
KAFSTV                             13
KVERTTIJD                           8
KREISDUUR                          11
ROLAUTO                             2
PARKEERKOSTEN                       0
VERTRAGING                          0
DAGSOORT                            1
OORDEEL_AUTO

In [807]:
print(df.columns)

Index(['HHID', 'PERSID', 'VPLID', 'RitID', 'HH_VALID', 'P_VALID', 'KHVM',
       'WEGGEWEEST', 'VERTREKP', 'AANTVPL', 'VPLDAGNR', 'VERPL', 'VERPLNR',
       'TOER', 'TOER_TYPE', 'AANTRIT', 'KMOTIEF', 'VERTPROV', 'AANKPROV',
       'KAFSTV', 'KVERTTIJD', 'KREISDUUR', 'ROLAUTO', 'PARKEERKOSTEN',
       'VERTRAGING', 'DAGSOORT', 'OORDEEL_AUTO', 'OORDEEL_TREIN',
       'OORDEEL_BTM', 'OORDEEL_FIETS', 'OORDEEL_BROMMER', 'OORDEEL_LOPEN',
       'GEBRUIK_AUTO_STELLING3', 'GEBRUIK_AUTO_STELLING4',
       'GEBRUIK_TREIN_STELLING3', 'GEBRUIK_TREIN_STELLING4',
       'GEBRUIK_BTM_STELLING3', 'GEBRUIK_BTM_STELLING4',
       'GEBRUIK_FIETS_STELLING2', 'GEBRUIK_FIETS_STELLING3',
       'GEBRUIK_LOPEN_STELLING3', 'GEBRUIK_LOPEN_STELLING4', 'HHPERS', 'HHSAM',
       'N_KIND', 'HHBRUTOINK2_w5', 'HHAUTO_N', 'HHAUTO', 'HHBESTEL',
       'HHHYBRID', 'HHMOTOR', 'HHBROM', 'HHSNOR', 'HHFIETS', 'HHVOUWFIETS',
       'HHEBIKE'],
      dtype='object')


### График 1 (Целевая переменная (KHVM))

In [808]:
df2 = df_map.groupby('KHVM').size().reset_index().rename(columns = {'KHVM': 'Транспорт', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
print(df2)

fig2 = px.bar(df2, x='Транспорт', y = 'Количество поездок', text_auto=True)
fig2.update_layout(title = {'text': 'Распределение количества использования конкретного транспортного средства', 'x': 0.5})
fig2.show()

          Транспорт  Количество поездок
2     Car as driver                1508
6             Train                 834
1    Bus/tram/metro                 746
0           Bicycle                 629
3  Car as passenger                 424
4             Other                 193
7           Walking                 129
5     Scooter/moped                  83


### График 2 (Основные характеристики поездок)

In [809]:
df1 = df_map.groupby(['KAFSTV', 'KHVM']).size().reset_index().rename(columns = {'KAFSTV': 'Расстояние поездки', 0: 'Количество поездок', 'KHVM': 'Транспортное средство'})
print(df1)

fig1 = px.bar(df1, 
              x='Расстояние поездки', 
              y = 'Количество поездок',
              color='Транспортное средство', 
              text_auto=True,
              width=1200,
              height=800)
fig1.update_layout(title = {'text': 'Распределение количества поездок по расстоянию и используемому виду транспорта', 'x': 0.5})
fig1.show()

   Расстояние поездки Транспортное средство  Количество поездок
0            10-15 km               Bicycle                  50
1            10-15 km        Bus/tram/metro                 127
2            10-15 km         Car as driver                 182
3            10-15 km      Car as passenger                  66
4            10-15 km                 Other                  21
..                ...                   ...                 ...
69             >50 km        Bus/tram/metro                  29
70             >50 km         Car as driver                 125
71             >50 km      Car as passenger                  63
72             >50 km                 Other                  48
73             >50 km                 Train                 351

[74 rows x 3 columns]


**вывод для графика** ----

### График 3 (Влияние оценки транспорта (OORDEEL_*) на предпочтение (KHVM))

'OORDEEL_AUTO': 'оценка_автомобиля',
'OORDEEL_TREIN': 'оценка_поезда',
'OORDEEL_BTM': 'оценка_общественного_транспорта',
'OORDEEL_FIETS': 'оценка_велосипеда',
'OORDEEL_BROMMER': 'оценка_мопеда',
'OORDEEL_LOPEN': 'оценка_передвижения_пешком',

In [810]:
df31 = df.groupby('OORDEEL_AUTO').size().reset_index().rename(columns = {'OORDEEL_AUTO': 'Оценка автомобиля', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df31['Тип передвижения'] = 'Автомобиль'


df32 = df.groupby('OORDEEL_TREIN').size().reset_index().rename(columns = {'OORDEEL_TREIN': 'Оценка поезда', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df32['Тип передвижения'] = 'Поезд'


df33 = df.groupby('OORDEEL_BTM').size().reset_index().rename(columns = {'OORDEEL_BTM': 'Оценка общественного транспорта', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df33['Тип передвижения'] = 'Общественный транспорт'


df34 = df.groupby('OORDEEL_FIETS').size().reset_index().rename(columns = {'OORDEEL_FIETS': 'Оценка велосипеда', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df34['Тип передвижения'] = 'Велосипед'


df35 = df.groupby('OORDEEL_BROMMER').size().reset_index().rename(columns = {'OORDEEL_BROMMER': 'Оценка мотоцикла', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df35['Тип передвижения'] = 'Мотоцикл'


df36 = df.groupby('OORDEEL_LOPEN').size().reset_index().rename(columns = {'OORDEEL_LOPEN': 'Оценка ходить пешком', 0: 'Количество поездок'}).sort_values('Количество поездок', ascending=False)
df36['Тип передвижения'] = 'Пешая прогулка'


df3 = pd.concat([df31, df32, df33, df34, df35, df36])

overall = sum(df31['Количество поездок'])



df_long = pd.melt(
    df3,
    id_vars=['Количество поездок', 'Тип передвижения'],
    value_vars=['Оценка автомобиля', 'Оценка поезда', 'Оценка общественного транспорта',
                'Оценка велосипеда', 'Оценка мотоцикла', 'Оценка ходить пешком'],
    var_name='Тип оценки',
    value_name='Оценка'
)


df_long.dropna(inplace=True)

df_long['Доля'] = round((df_long['Количество поездок'] / overall) * 100, 2)


df_long['Оценка'] = df_long['Оценка'].replace({1: 'Очень негативно',
                                   2: 'Негативно',
                                   3: 'Нейтрально',
                                   4: 'Позитивно',
                                   5: 'Очень позитивно',
                                   6: 'Нет оценки',
                                   })
order = ['Очень позитивно', 'Позитивно', 'Нейтрально', 'Очень негативно', 'Негативно', 'Нет оценки']
df_long['Оценка'] = pd.Categorical(df_long['Оценка'], categories=order, ordered=True)

df_long = df_long.rename(columns={'Количество поездок': 'Количество ответивших'})

#--------------ОТОБРАЗИМ ВСЕ В КОЛИЧЕСТВАХ----------------------

fig3 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Количество ответивших', 
    text='Количество ответивших',
    facet_col='Оценка',
    facet_col_wrap=3,  # Создаем две строки по 3 графика
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig3.update_traces(textfont_size=10, textposition='outside', cliponaxis=False)

fig3.update_layout(
    title={'text': 'Распределение количества мнений респондентов по поводу использования транспорта по оценкам', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    legend=dict(title='Тип передвижения', orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5)
)

fig3.show()


#--------------ОТОБРАЗИМ ВСЕ В ДОЛЯХ----------------------
fig4 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Доля', 
    text='Доля',
    facet_col='Оценка',
    facet_col_wrap=3,
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig4.update_traces(
    textposition='outside',
    textfont=dict(size=12),  
    cliponaxis=False,  
    texttemplate='%{text}%', 
    offsetgroup=1 
)

fig4.update_layout(
    title={'text': 'Распределение количества мнений респондентов по поводу использования транспорта по оценкам (доли)', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    uniformtext_minsize=12,
    uniformtext_mode='show'  
)

fig4.show()


### График 4 (Восприятие экономии времени разных видов транспорта)

    # Восприятие транспорта (важно для моделирования предпочтений)
    'GEBRUIK_AUTO_STELLING3': 'восприятие_экономии_времени_авто',
    'GEBRUIK_AUTO_STELLING4': 'восприятие_безопасности_авто',
    'GEBRUIK_TREIN_STELLING3': 'восприятие_экономии_времени_поезда',
    'GEBRUIK_TREIN_STELLING4': 'восприятие_безопасности_поезда',
    'GEBRUIK_BTM_STELLING3': 'восприятие_экономии_времени_общественного_транспорта',
    'GEBRUIK_BTM_STELLING4': 'восприятие_безопасности_общественного_транспорта',
    'GEBRUIK_FIETS_STELLING2': 'восприятие_релаксации_на_велосипеде',
    'GEBRUIK_FIETS_STELLING3': 'восприятие_экономии_времени_велосипеда',
    'GEBRUIK_LOPEN_STELLING3': 'восприятие_экономии_времени_ходьбы',
    'GEBRUIK_LOPEN_STELLING4': 'восприятие_безопасности_ходьбы',

In [811]:
df51 = df.groupby('GEBRUIK_AUTO_STELLING3').size().reset_index()

df51 = df51.rename(columns = {'GEBRUIK_AUTO_STELLING3': 'Оценка экономии времени на автомобиле', 
                              0: 'Количество ответивших'})

df51.sort_values('Количество ответивших', ascending=False)

df51['Тип передвижения'] = 'Автомобиль'




df52 = df.groupby('GEBRUIK_TREIN_STELLING3').size().reset_index()

df52 = df52.rename(columns = {'GEBRUIK_TREIN_STELLING3': 'Оценка экономии времени на поезде', 
                              0: 'Количество ответивших'})

df52.sort_values('Количество ответивших', ascending=False)
df52['Тип передвижения'] = 'Поезд'


df53 = df.groupby('GEBRUIK_BTM_STELLING3').size().reset_index()

df53 = df53.rename(columns={'GEBRUIK_BTM_STELLING3': 'Оценка экономии времени на общественном транспорте', 
                            0: 'Количество ответивших'})

df53 = df53.sort_values('Количество ответивших', ascending=False)
df53['Тип передвижения'] = 'Общественный транспорт'



df54 = df.groupby('GEBRUIK_FIETS_STELLING3').size().reset_index()
df54 = df54.rename(columns={'GEBRUIK_FIETS_STELLING3': 'Оценка экономии времени на велосипеде', 
                            0: 'Количество ответивших'})
df54 = df54.sort_values('Количество ответивших', ascending=False)
df54['Тип передвижения'] = 'Велосипед'


df55 = df.groupby('GEBRUIK_LOPEN_STELLING3').size().reset_index()
df55 = df55.rename(columns={'GEBRUIK_LOPEN_STELLING3': 'Оценка экономии времени при ходьбе', 
                            0: 'Количество ответивших'})
df55 = df55.sort_values('Количество ответивших', ascending=False)
df55['Тип передвижения'] = 'Ходьба'




df5 = pd.concat([df51, df52, df53, df54, df55])

overall = sum(df51['Количество ответивших'])


print(overall)

df_long = pd.melt(
    df5,
    id_vars=['Количество ответивших', 'Тип передвижения'],
    value_vars=['Оценка экономии времени на автомобиле', 'Оценка экономии времени на поезде', 'Оценка экономии времени на общественном транспорте', 'Оценка экономии времени на велосипеде', 'Оценка экономии времени при ходьбе'],
    var_name='Тип оценки',
    value_name='Оценка'
)

# Удаление NaN значений
df_long.dropna(inplace=True)

# Вычисление долей в процентах
df_long['Доля'] = round((df_long['Количество ответивших'] / overall) * 100, 2)

# Замена категорий оценок
df_long['Оценка'] = df_long['Оценка'].replace({
    1: 'Абсолютно не согласен',
    2: 'Не согласен',
    3: 'Не уверен',
    4: 'Согласен',
    5: 'Абсолютно согласен',
    6: 'Нет оценки'
})

# Задание порядка категорий
order = ['Абсолютно согласен', 'Согласен', 'Не уверен', 'Абсолютно не согласен', 'Не согласен', 'Нет оценки']
df_long['Оценка'] = pd.Categorical(df_long['Оценка'], categories=order, ordered=True)

# Построение графика для количества поездок
fig3 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Количество ответивших', 
    text='Количество ответивших',
    facet_col='Оценка',
    facet_col_wrap=3,
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig3.update_traces(textfont_size=10, textposition='outside', cliponaxis=False)

fig3.update_layout(
    title={'text': 'Согласны ли вы, что этот вид транспорта экономит время?', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    legend=dict(title='Тип передвижения', orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5)
)

fig3.show()



#--------------ОТОБРАЗИМ ВСЕ В ДОЛЯХ----------------------
fig4 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Доля', 
    text='Доля',
    facet_col='Оценка',
    facet_col_wrap=3,
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig4.update_traces(
    textposition='outside',
    textfont=dict(size=12),  
    cliponaxis=False,  
    texttemplate='%{text}%', 
    offsetgroup=1 
)

fig4.update_layout(
    title={'text': 'Распределение использования транспорта по оценкам', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    uniformtext_minsize=12,
    uniformtext_mode='show'  
)

fig4.show()


4546


### График 5 (Восприятие безопасности разных видов транспорта)

In [812]:
df51 = df.groupby('GEBRUIK_AUTO_STELLING4').size().reset_index()

df51 = df51.rename(columns = {'GEBRUIK_AUTO_STELLING4': 'Оценка безопасности автомобиля', 
                              0: 'Количество ответивших'})

df51.sort_values('Количество ответивших', ascending=False)

df51['Тип передвижения'] = 'Автомобиль'




df52 = df.groupby('GEBRUIK_TREIN_STELLING4').size().reset_index()

df52 = df52.rename(columns = {'GEBRUIK_TREIN_STELLING4': 'Оценка безопасности поезда', 
                              0: 'Количество ответивших'})

df52.sort_values('Количество ответивших', ascending=False)
df52['Тип передвижения'] = 'Поезд'


df53 = df.groupby('GEBRUIK_BTM_STELLING4').size().reset_index()

df53 = df53.rename(columns={'GEBRUIK_BTM_STELLING4': 'Оценка безопасности общественного транспорта', 
                            0: 'Количество ответивших'})

df53 = df53.sort_values('Количество ответивших', ascending=False)
df53['Тип передвижения'] = 'Общественный транспорт'




df54 = df.groupby('GEBRUIK_LOPEN_STELLING4').size().reset_index()
df54 = df54.rename(columns={'GEBRUIK_LOPEN_STELLING4': 'Оценка безопасности ходьбы', 
                            0: 'Количество ответивших'})
df54 = df54.sort_values('Количество ответивших', ascending=False)
df54['Тип передвижения'] = 'Ходьба'



df5 = pd.concat([df51, df52, df53, df54])

overall = sum(df51['Количество ответивших'])


print(overall)

df_long = pd.melt(
    df5,
    id_vars=['Количество ответивших', 'Тип передвижения'],
    value_vars=['Оценка безопасности автомобиля', 'Оценка безопасности поезда', 'Оценка безопасности общественного транспорта', 'Оценка безопасности ходьбы'],
    var_name='Тип оценки',
    value_name='Оценка'
)

# Удаление NaN значений
df_long.dropna(inplace=True)

# Вычисление долей в процентах
df_long['Доля'] = round((df_long['Количество ответивших'] / overall) * 100, 2)

# Замена категорий оценок
df_long['Оценка'] = df_long['Оценка'].replace({
    1: 'Абсолютно не согласен',
    2: 'Не согласен',
    3: 'Не уверен',
    4: 'Согласен',
    5: 'Абсолютно согласен',
    6: 'Нет оценки'
})

# Задание порядка категорий
order = ['Абсолютно согласен', 'Согласен', 'Не уверен', 'Абсолютно не согласен', 'Не согласен', 'Нет оценки']
df_long['Оценка'] = pd.Categorical(df_long['Оценка'], categories=order, ordered=True)

# Построение графика для количества поездок
fig3 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Количество ответивших', 
    text='Количество ответивших',
    facet_col='Оценка',
    facet_col_wrap=3,
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig3.update_traces(textfont_size=10, textposition='outside', cliponaxis=False)

fig3.update_layout(
    title={'text': 'Согласны ли вы, что этот вид транспорта безопасен?', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    legend=dict(title='Тип передвижения', orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5)
)

fig3.show()



#--------------ОТОБРАЗИМ ВСЕ В ДОЛЯХ----------------------
fig4 = px.bar(
    df_long, 
    x='Тип передвижения', 
    y='Доля', 
    text='Доля',
    facet_col='Оценка',
    facet_col_wrap=3,
    height=600, 
    width=1400,
    category_orders={'Оценка': order}
)

fig4.update_traces(
    textposition='outside',
    textfont=dict(size=12),  
    cliponaxis=False,  
    texttemplate='%{text}%', 
    offsetgroup=1 
)

fig4.update_layout(
    title={'text': 'Распределение восприятия безопасности транспорта по оценкам', 'x': 0.5},
    margin=dict(t=60, b=40, l=40, r=40),
    uniformtext_minsize=12,
    uniformtext_mode='show'  
)

fig4.show()


4546


### График 6 (Распределение количетсва разныъ семей с разным финансовым положением)

In [813]:
['HHAUTO_N', 'HHFIETS', 'HHEBIKE', 'HHPERS', 'N_KIND']

['HHAUTO_N', 'HHFIETS', 'HHEBIKE', 'HHPERS', 'N_KIND']

In [814]:
df6 = df_map[['HHBRUTOINK2_w5', 'HHSAM', 'HHPERS', 'N_KIND', 'KHVM']]

df6.rename(columns = {'HHBRUTOINK2_w5': 'Доход семьи',
                      'HHSAM': 'Категория семьи',
                      'HHPERS': 'Количество человек в семье',
                      'N_KIND': 'Количетсво детей в семье',
                      'KHVM': 'Средство передвижения'}, inplace=True)

df61 = df6.groupby(['Доход семьи', 'Категория семьи']).size().reset_index()
df61['Доход семьи'] = df61['Доход семьи'].str.replace('\x80', '€', regex=False)


df61.rename(columns = {0: 'Количество'}, inplace=True)

fig6 = px.bar(df61, 
              x = 'Категория семьи',
              y = 'Количество',
              color='Доход семьи',
              text_auto=True)
fig6.update_layout(title = {'text': 'Распределение категорий семей и их заработная плата', 'x': 0.5})
fig6.show()



### График 7 (Использование средств передвижения разными семьями и разными финансовыми положениями)

In [815]:
# Группируем данные только ОДИН раз
df62 = df6.groupby(['Доход семьи', 'Категория семьи', 'Средство передвижения']).size().reset_index()

# 1. Сначала делаем замену символов
df62['Доход семьи'] = df62['Доход семьи'].str.replace('[\x80\xad]', '€', regex=True)

# 2. Затем применяем словарь замен
income_mapping = {
    'minimum (< € 13,700)': '<€13,700',
    'below the national benchmark income (€ 13,700 - < € 28,600)': '€13,700 - <€28,600',
    'national benchmark income (€ 28,600 - < € 42,400)': '€28,600 - <€42,400',
    '1-2x the national benchmark income (€ 42,400 - < € 71,000)': '€42,400 - <€71,000',
    '2x the national benchmark income (€ 71,000 - < € 84,700)': '€71,000 - <€84,700',
    'more than 2x the national benchmark income (>= € 84,700)': '>€84,700',
    'Do not know / do not want to say': 'Не захотел отвечать'
}

df62['Доход семьи'] = df62['Доход семьи'].replace(income_mapping)


df62.rename(columns={0: 'Количество'}, inplace=True)

df62 = df62.dropna(subset=['Доход семьи'])

income_order = [
    '<€13,700',
    '€13,700 - <€28,600',
    '€28,600 - <€42,400',
    '€42,400 - <€71,000',
    '€71,000 - <€84,700',
    '>€84,700',
    'Не захотел отвечать'
]

df62['Доход семьи'] = pd.Categorical(
    df62['Доход семьи'], 
    categories=income_order,
    ordered=True
)

# 6. Сортируем данные согласно категориям
df62 = df62.sort_values('Доход семьи')

fig6 = px.bar(
    df62, 
    x='Категория семьи',
    y='Количество',
    facet_col='Доход семьи',
    color='Средство передвижения',
    text_auto=True,
    height=600, 
    width=1800,
    category_orders={'Доход семьи': income_order}
)


fig6.update_layout(
    title={'text': 'Распределение категорий семей по доходу и транспорту', 'x': 0.5, 'y': 0.95},
    margin=dict(t=150, b=20),  
    legend=dict(
        title='Транспорт',
        orientation='h',
        yanchor='bottom',
        y=1.25,  # Опускаем легенду ниже
        xanchor='center',
        x=0.5
    ),
    xaxis={'visible': True, 'title': None}, 
)


fig6.update_xaxes(
    title_text='',  
    showticklabels=True,  
    tickangle=-45 
)


for annotation in fig6.layout.annotations:
    annotation.text = annotation.text.split("=")[1]

fig6.show()

### График 8 (Матрица корреляции)

**1. Целевой признак:**
- **`KHVM` — Основной транспорт.**  
  - Основной фокус исследования. Цель — выявить факторы, влияющие на выбор транспорта.

---

**2. Характеристики поездок:**
- **`AANTVPL` — Количество поездок.**  
  - Частота поездок может быть связана с выбором основного транспорта.

- **`KAFSTV` — Класс расстояния поездки.**  
  - Дальние поездки могут стимулировать выбор более быстрого транспорта.

- **`KMOTIEF` — Цель поездки.**  
  - Цели поездок (работа, отдых, покупки и т.д.) влияют на выбор транспорта.

- **`TOER` — Флаг круговой поездки.**  
  - Особенности круговых поездок могут влиять на транспортные предпочтения.

- **`ROLAUTO` — Роль в автомобиле (водитель/пассажир).**  
  - Роль влияет на восприятие комфорта и затрат.

- **`DAGSOORT` — Тип дня (будний/выходной).**  
  - В выходные дни транспортные предпочтения могут отличаться.

---

**3. Оценки видов транспорта:**
- **`OORDEEL_AUTO` — Оценка автомобиля.**  
- **`OORDEEL_TREIN` — Оценка поезда.**  
- **`OORDEEL_BTM` — Оценка общественного транспорта.**  
- **`OORDEEL_FIETS` — Оценка велосипеда.**  
- **`OORDEEL_BROMMER` — Оценка мопеда.**  
- **`OORDEEL_LOPEN` — Оценка ходьбы.**  

> Субъективные оценки различных видов транспорта помогают выявить предпочтения и потенциальные барьеры для использования.

---

**4. Восприятие транспорта:**
- **`GEBRUIK_AUTO_STELLING3` / `GEBRUIK_AUTO_STELLING4` — Восприятие авто (экономия времени, безопасность).**  
- **`GEBRUIK_TREIN_STELLING3` / `GEBRUIK_TREIN_STELLING4` — Восприятие поезда.**  
- **`GEBRUIK_BTM_STELLING3` / `GEBRUIK_BTM_STELLING4` — Восприятие общественного транспорта.**  
- **`GEBRUIK_FIETS_STELLING2` / `GEBRUIK_FIETS_STELLING3` — Восприятие велосипеда.**  
- **`GEBRUIK_LOPEN_STELLING3` / `GEBRUIK_LOPEN_STELLING4` — Восприятие ходьбы.**  

> Восприятие транспорта по ключевым параметрам (экономия времени, безопасность, удобство) напрямую связано с вероятностью его использования.

---

**5. Социально-экономические характеристики домохозяйства:**
- **`HHBRUTOINK2_w5` — Класс дохода.**  
  - Доход влияет на доступность и выбор транспорта.

- **`HHPERS` — Количество людей в домохозяйстве.**  
  - Большее количество людей может стимулировать использование более вместительных или экономичных транспортных средств.

- **`HHAUTO`, `HHBESTEL`, `HHHYBRID`, `HHMOTOR`, `HHBROM`, `HHSNOR`, `HHFIETS`, `HHVOUWFIETS`, `HHEBIKE` — Наличие транспортных средств.**  
  - Наличие автомобилей, велосипедов и других транспортных средств — это очевидные предикторы выбора основного вида транспорта.

  

In [816]:
# Выбираем только те признаки, которые могут быть связаны с KHVM
columns_of_interest = [
    'KHVM', 'AANTVPL', 'KAFSTV', 'KMOTIEF', 'TOER', 'ROLAUTO',
    'DAGSOORT', 'OORDEEL_AUTO', 'OORDEEL_TREIN', 'OORDEEL_BTM', 'OORDEEL_FIETS', 'OORDEEL_BROMMER', 'OORDEEL_LOPEN',
    'GEBRUIK_AUTO_STELLING3', 'GEBRUIK_AUTO_STELLING4',
    'GEBRUIK_TREIN_STELLING3', 'GEBRUIK_TREIN_STELLING4',
    'GEBRUIK_BTM_STELLING3', 'GEBRUIK_BTM_STELLING4',
    'GEBRUIK_FIETS_STELLING2', 'GEBRUIK_FIETS_STELLING3',
    'GEBRUIK_LOPEN_STELLING3', 'GEBRUIK_LOPEN_STELLING4',
    'HHBRUTOINK2_w5', 'HHAUTO', 'HHPERS', 'HHBESTEL', 
    'HHHYBRID',
    'HHMOTOR',
    'HHBROM',
    'HHSNOR',
    'HHFIETS',
    'HHVOUWFIETS',
    'HHEBIKE'
]


df_corr = df[columns_of_interest]

corr_matrix = df_corr.corr()


fig = px.imshow(
    corr_matrix, 
    height=1000, 
    width=1000, 
    text_auto='.2f', 
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)

fig.update_layout(
    title = {'text': 'Матрица корреляций признаков, связанных с основным видом транспорта (KHVM)', 'x': 0.5},
    margin=dict(t=100, b=50, l=50, r=50),
    font=dict(size=14) 
)

fig.show()


# Разделение на выборки

Так как у нас задача классификации, то таргетом будет у нас `KHVM`, средство передвижения, кроме того, у нас несбалансированны классы, поэтому возьмем отношение 70/30 (тестовые/тренировочные данные) и выберем метод разбиения на стратифицированные выборки 

In [817]:
X = df.drop(columns=['KHVM'])
y = df['KHVM']

X_traim, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, stratify=y, random_state=42, shuffle=True)


# Генерирование признаков

In [818]:
# отношение количества поездок к количеству сегментов поездки. Такой признак может дать представление о том, насколько часто человек делит свои поездки на отдельные сегменты.
df['AANTVPL_AANTRIT_RATIO'] = df['AANTVPL'] / df['AANTRIT']

# средняя оценка транспорта
transport_columns = ['OORDEEL_AUTO', 'OORDEEL_TREIN', 'OORDEEL_BTM', 'OORDEEL_FIETS', 'OORDEEL_BROMMER', 'OORDEEL_LOPEN']

df['AVG_TRANSPORT_RATING'] = df[transport_columns].groupby(df['PERSID']).transform('mean').mean(axis=1)



# уникальные ТС у человека

df['NUM_UNIQUE_TRANSPORT'] = df.groupby('PERSID')['KHVM'].transform('nunique')



# Флаг о том, что человек не покидвет свою провинцию

def flag_department(df):
    if df['AANKPROV'] != df['VERTPROV']:
        df['SAME_DEPARTMENT'] = 1
    else:
        df['SAME_DEPARTMENT'] = 0
    return df['SAME_DEPARTMENT']

df['SAME_DEPARTMENT'] = df.apply(flag_department, axis = 1)


# Отношение людей к количеству авто

df['CAR_TO_PEOPLE_RATIO'] = df['HHAUTO_N'] / df['HHPERS']

Проверим теперь эффективность наших признаков на матрице корреляции

In [819]:
# Выбираем только те признаки, которые могут быть связаны с KHVM
columns_of_interest = [
    'KHVM', 'AANTVPL', 'KAFSTV', 'KMOTIEF', 'TOER', 'ROLAUTO',
    'DAGSOORT', 'OORDEEL_AUTO', 'OORDEEL_TREIN', 'OORDEEL_BTM', 'OORDEEL_FIETS', 'OORDEEL_BROMMER', 'OORDEEL_LOPEN',
    'GEBRUIK_AUTO_STELLING3', 'GEBRUIK_AUTO_STELLING4',
    'GEBRUIK_TREIN_STELLING3', 'GEBRUIK_TREIN_STELLING4',
    'GEBRUIK_BTM_STELLING3', 'GEBRUIK_BTM_STELLING4',
    'GEBRUIK_FIETS_STELLING2', 'GEBRUIK_FIETS_STELLING3',
    'GEBRUIK_LOPEN_STELLING3', 'GEBRUIK_LOPEN_STELLING4',
    'HHBRUTOINK2_w5', 'HHAUTO', 'HHPERS', 'HHBESTEL', 
    'HHHYBRID',
    'HHMOTOR',
    'HHBROM',
    'HHSNOR',
    'HHFIETS',
    'HHVOUWFIETS',
    'HHEBIKE',
    'AANTVPL_AANTRIT_RATIO', 'AVG_TRANSPORT_RATING', 'NUM_UNIQUE_TRANSPORT', 'SAME_DEPARTMENT', 'CAR_TO_PEOPLE_RATIO'
]


df_corr = df[columns_of_interest]

corr_matrix = df_corr.corr()


fig = px.imshow(
    corr_matrix, 
    height=1000, 
    width=1000, 
    text_auto='.2f',  
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1
)

fig.update_layout(
    title = {'text': 'Матрица корреляций с новыми признаками', 'x': 0.5},
    margin=dict(t=100, b=50, l=50, r=50),
    font=dict(size=14) 
)

fig.show()


В итоге получилась слабая корреляция (от -0.2 до 0.2) для новых фичей. Но мы все равно эти фичи отсавляем, так как любые признаки могут нам пригодиться, так как модели могут все равно извлечь полезную информацию из таких признаков, особенно если используемые алгоритмы достаточно сложные